In [1]:
import pandas as pd
import numpy as np
import pathlib
import re
from datetime import datetime
from arrests_columns import ArrestsColumns

In [2]:
proc_dir = pathlib.Path('./proc_data/')
d = pd.read_parquet(proc_dir / 'arrests.parquet')
use_cols = [v.value[0] for k, v in ArrestsColumns.__members__.items() if not v.value[2] and not v.value[3]]
print(f'{use_cols=}')
d = d[use_cols].copy()
d.head()

use_cols=['Apprehension Date', 'Apprehension State', 'Apprehension AOR', 'Final Program', 'Apprehension Method', 'Apprehension Criminality', 'Case Status', 'Case Category', 'Departed Date', 'Departure Country', 'Final Order Date', 'Birth Year', 'Citizenship Country', 'Gender', 'Apprehension Site Landmark', 'Unique Identifier']


,Apprehension Date,Apprehension State,Apprehension AOR,Final Program,Apprehension Method,Apprehension Criminality,Case Status,Case Category,Departed Date,Departure Country,Final Order Date,Birth Year,Citizenship Country,Gender,Apprehension Site Landmark,Unique Identifier
0,2025-07-26 18:50:27,TEXAS,Houston Area of Responsibility,ERO Criminal Alien Program,Custodial Arrest,3 Other Immigration Violator,ACTIVE,[1A] Voluntary Departure - Un-Expired and Un-E...,NaT,None,NaT,2002,HONDURAS,Male,"MTG GENERAL AREA, NON-SPECIFIC",000066a0cf0da888503506a0bd39e0772fada2ce
1,2025-07-02 17:57:46,NEVADA,Salt Lake City Area of Responsibility,ERO Criminal Alien Program,CAP Local Incarceration,2 Pending Criminal Charges,ACTIVE,[8B] Excludable / Inadmissible - Under Adjudic...,NaT,None,NaT,1984,MEXICO,Male,NORTH LAS VEGAS DETENTION CENTER,0000718e93d4f75cdec49ccdd21bf7a09f48a8b7
2,2024-07-05 10:22:00,NEW YORK,New York City Area of Responsibility,ERO Criminal Alien Program,CAP State Incarceration,1 Convicted Criminal,6-Deported/Removed - Deportability,[3] Deportable - Administratively Final Order,2024-09-05,GUYANA,2014-03-13,1968,GUYANA,Female,CAP - BEDFORD CORRECTIONAL FACILITY NY STATE,0000f265b1ff8abbfbec54888dd2cddb62e21625
3,2025-05-28 07:42:11,TEXAS,San Antonio Area of Responsibility,ERO Criminal Alien Program,CAP Federal Incarceration,1 Convicted Criminal,8-Excluded/Removed - Inadmissibility,[8F] Expedited Removal,2025-06-25,COLOMBIA,2025-03-02,2004,COLOMBIA,Male,"DLR GENERAL AREA, NON-SPECIFIC",00010f535759072a0dfa031f9f3fc5774e49ccee
4,2024-09-02 23:22:00,TEXAS,Houston Area of Responsibility,ERO Criminal Alien Program,CAP Local Incarceration,1 Convicted Criminal,8-Excluded/Removed - Inadmissibility,[8C] Excludable / Inadmissible - Administrativ...,2025-02-12,EL SALVADOR,2025-01-10,2000,EL SALVADOR,Male,"HARRIS COUNTY JAIL, HOUSTON, TX",000143a0747f494567db683e9c21f16fbd6052b0


In [3]:
def missing_percent_print(d, col_name):
    print(f"{col_name}, {(d[col_name].isnull().sum() / len(d)).item() * 100:.4}% missing values")

In [4]:
# repeat the same country string replacements

def update_country_strings(df, col_name):
    df[col_name] = df[col_name].replace('PALESTINE BORN BEFORE 1948', 'PALESTINE')
    df[col_name] = df[col_name].replace('Palestine', 'PALESTINE')
    
    # Soveirgn states and territories
    df[col_name] = df[col_name].replace('CHINA, PEOPLES REPUBLIC OF', 'CHINA')
    df[col_name] = df[col_name].replace('HONG KONG', 'CHINA')
    df[col_name] = df[col_name].replace('MACAU', 'CHINA')
    df[col_name] = df[col_name].replace('BRITISH VIRGIN ISLANDS', 'UNITED KINGDOM')
    df[col_name] = df[col_name].replace('BERMUDA', 'UNITED KINGDOM')
    df[col_name] = df[col_name].replace('NETHERLANDS ANTILLES', 'NETHERLANDS')
    df[col_name] = df[col_name].replace('SAINT MARTIN(FRENCH)', 'FRANCE')
    df[col_name] = df[col_name].replace('FRENCH POLYNESIA', 'FRANCE')
    df[col_name] = df[col_name].replace('GUADELOUPE', 'FRANCE')
    df[col_name] = df[col_name].replace('SAINT BARTHELEMY', 'FRANCE')
    df[col_name] = df[col_name].replace('NEW CALEDONIA', 'FRANCE')
    df[col_name] = df[col_name].replace('REUNION', 'FRANCE')
    df[col_name] = df[col_name].replace('FRENCH GUIANA', 'FRANCE')
    df[col_name] = df[col_name].replace('SINT MAARTEN(DUTCH)', 'NETHERLANDS')
    df[col_name] = df[col_name].replace('ARUBA', 'NETHERLANDS')
    df[col_name] = df[col_name].replace('SINT EUSTATIUS', 'NETHERLANDS')
    df[col_name] = df[col_name].replace('SABA', 'NETHERLANDS')
    df[col_name] = df[col_name].replace('BONAIRE', 'NETHERLANDS')
    df[col_name] = df[col_name].replace('CURACAO', 'NETHERLANDS')
    df[col_name] = df[col_name].replace('TURKS AND CAICOS ISLANDS', 'UNITED KINGDOM')
    df[col_name] = df[col_name].replace('CAYMAN ISLANDS', 'UNITED KINGDOM')
    df[col_name] = df[col_name].replace('MONTSERRAT', 'UNITED KINGDOM')
    df[col_name] = df[col_name].replace('ANGUILLA', 'UNITED KINGDOM')
    df[col_name] = df[col_name].replace('ST. HELENA', 'UNITED KINGDOM')
    df[col_name] = df[col_name].replace('CHRISTMAS ISLAND', 'AUSTRALIA')
    # note: "DEM REP OF CONGO" is a different country from "CONGO" which I assume is the republic of congo

update_country_strings(d, ArrestsColumns.CITIZENSHIP_COUNTRY.column_name)

missing_percent_print(d, ArrestsColumns.CITIZENSHIP_COUNTRY.column_name)

Citizenship Country, 0.0% missing values


In [5]:
missing_percent_print(d, ArrestsColumns.APPREHENSION_AOR.column_name)
# sorted(list(d[~d[ArrestsColumns.APPREHENSION_AOR.column_name].isna()][ArrestsColumns.APPREHENSION_AOR.column_name].unique()))

Apprehension AOR, 2.057% missing values


In [6]:
# this field includes U.S. territories, a state in mexico, and tags for arrests in "ARMED FORCES" areas outside the U.S.
missing_percent_print(d, ArrestsColumns.APPREHENSION_STATE.column_name)
# sorted(list(d[~d[ArrestsColumns.APPREHENSION_STATE.column_name].isna()][ArrestsColumns.APPREHENSION_STATE.column_name].unique()))

Apprehension State, 19.49% missing values


In [7]:
missing_percent_print(d, ArrestsColumns.FINAL_PROGRAM.column_name)

Final Program, 0.0% missing values


In [8]:
# under the same same program?
d[ArrestsColumns.FINAL_PROGRAM.column_name] = d[ArrestsColumns.FINAL_PROGRAM.column_name].replace('287G Program', '287g Task Force')
# https://www.ice.gov/identify-and-arrest/fugitive-operations
# another name for the "Mobile criminal apprehension teams"?
d[ArrestsColumns.FINAL_PROGRAM.column_name] = d[ArrestsColumns.FINAL_PROGRAM.column_name].replace('Mobile Criminal Alien Team', 'Fugitive Operations')
# "Violent Criminal Alien Section (VCAS)" is under the "Criminal Alien program"
d[ArrestsColumns.FINAL_PROGRAM.column_name] = d[ArrestsColumns.FINAL_PROGRAM.column_name].replace('Violent Criminal Alien Section', 'ERO Criminal Prosecutions')
# https://portal.ice.gov/immigration-guide/atd

d[ArrestsColumns.FINAL_PROGRAM.column_name].value_counts()

Final Program
ERO Criminal Alien Program             176755
Fugitive Operations                     47054
Non-Detained Docket Control             35299
Alternatives to Detention               13992
Detained Docket Control                  9676
287g Task Force                          5289
Juvenile                                  488
ERO Criminal Prosecutions                 307
Detention and Deportation                 269
Law Enforcement Area Response Units        65
Joint Criminal Alien Response Team          4
Name: count, dtype: int64

In [9]:
missing_percent_print(d, ArrestsColumns.APPREHENSION_METHOD.column_name)

Apprehension Method, 0.0% missing values


In [10]:
# filter out "Criminal Alien Program" completely because it's too generic
d = d[d[ArrestsColumns.APPREHENSION_METHOD.column_name] != 'Criminal Alien Program'].copy()
# assume several categories are the same
d[ArrestsColumns.APPREHENSION_METHOD.column_name] = d[ArrestsColumns.APPREHENSION_METHOD.column_name].replace('Transportation Check Bus', 'Traffic Check')
d[ArrestsColumns.APPREHENSION_METHOD.column_name] = d[ArrestsColumns.APPREHENSION_METHOD.column_name].replace('Transportation Check Passenger Train', 'Traffic Check')
d[ArrestsColumns.APPREHENSION_METHOD.column_name] = d[ArrestsColumns.APPREHENSION_METHOD.column_name].replace('Transportation Check Aircraft', 'Traffic Check')
d[ArrestsColumns.APPREHENSION_METHOD.column_name] = d[ArrestsColumns.APPREHENSION_METHOD.column_name].replace('Presented During Inspection', 'Inspections')
d[ArrestsColumns.APPREHENSION_METHOD.column_name] = d[ArrestsColumns.APPREHENSION_METHOD.column_name].replace('Crewman/Stowaway', 'Inspections')
# put remaining tiny categories into 'Other efforts' category
d[ArrestsColumns.APPREHENSION_METHOD.column_name] = d[ArrestsColumns.APPREHENSION_METHOD.column_name].replace('Boat Patrol', 'Other efforts')
d[ArrestsColumns.APPREHENSION_METHOD.column_name] = d[ArrestsColumns.APPREHENSION_METHOD.column_name].replace('Patrol Interior', 'Other efforts')
d[ArrestsColumns.APPREHENSION_METHOD.column_name] = d[ArrestsColumns.APPREHENSION_METHOD.column_name].replace('Other Task Force', 'Other efforts')
d[ArrestsColumns.APPREHENSION_METHOD.column_name] = d[ArrestsColumns.APPREHENSION_METHOD.column_name].replace('Other Agency (turned over to INS)', 'Other efforts')

d[ArrestsColumns.APPREHENSION_METHOD.column_name].value_counts()

Apprehension Method
CAP Local Incarceration                        119433
Non-Custodial Arrest                            64364
Located                                         34872
CAP Federal Incarceration                       24847
Other efforts                                   10921
CAP State Incarceration                         10875
ERO Reprocessed Arrest                           9328
287(g) Program                                   6922
Probation and Parole                             3920
Custodial Arrest                                 1490
Law Enforcement Agency Response Unit              974
Patrol Border                                     510
Worksite Enforcement                              385
Inspections                                       127
Traffic Check                                      92
Anti-Smuggling                                     83
Organized Crime Drug Enforcement Task Force        51
Name: count, dtype: int64

In [11]:
missing_percent_print(d, ArrestsColumns.APPREHENSION_CRIMINALITY.column_name)

Apprehension Criminality, 0.0% missing values


In [12]:
d[ArrestsColumns.APPREHENSION_CRIMINALITY.column_name].value_counts()

Apprehension Criminality
1 Convicted Criminal            132193
3 Other Immigration Violator     82034
2 Pending Criminal Charges       74967
Name: count, dtype: int64

In [13]:
missing_percent_print(d, ArrestsColumns.CASE_STATUS.column_name)

Case Status, 1.41% missing values


In [14]:
# these have different codes but different meanings?
# so it's probably find to combine them?
d[ArrestsColumns.CASE_STATUS.column_name] = d[ArrestsColumns.CASE_STATUS.column_name].replace('Z-SAW - Permanent Residence Granted', 'L-Legalization - Permanent Residence Granted')

d[ArrestsColumns.CASE_STATUS.column_name].value_counts()

Case Status
ACTIVE                                          113089
8-Excluded/Removed - Inadmissibility            110990
6-Deported/Removed - Deportability               40325
3-Voluntary Departure Confirmed                  10497
9-VR Witnessed                                    4449
E-Charging Document Canceled by ICE               2608
A-Proceedings Terminated                          1940
B-Relief Granted                                   902
L-Legalization - Permanent Residence Granted       111
5-Title 50 Expulsion                                92
0-Withdrawal Permitted - I-275 Issued               71
7-Died                                              41
Name: count, dtype: int64

In [15]:
missing_percent_print(d, ArrestsColumns.CASE_CATEGORY.column_name)

Case Category, 1.41% missing values


In [16]:
d[ArrestsColumns.CASE_CATEGORY.column_name].value_counts()

Case Category
[8C] Excludable / Inadmissible - Administrative Final Order Issued             87134
[16] Reinstated Final Order                                                    62531
[8B] Excludable / Inadmissible - Under Adjudication by IJ                      54165
[8A] Excludable / Inadmissible - Hearing Not Commenced                         18229
[3] Deportable - Administratively Final Order                                  14221
[8F] Expedited Removal                                                          9610
[8G] Expedited Removal - Credible Fear Referral                                 9208
[2A] Deportable - Under Adjudication by IJ                                      8513
[11] Administrative Deportation / Removal                                       6329
[9] VR Under Safeguards                                                         4632
[1A] Voluntary Departure - Un-Expired and Un-Extended Departure Period          2805
[8K] Expedited Removal Terminated due to Credible F

In [17]:
# consistent date filtering
d = d[d[ArrestsColumns.DEPARTED_DATE.column_name].isna() |
      ((pd.to_datetime('2023-9-1') <= d[ArrestsColumns.DEPARTED_DATE.column_name]) & 
      (d[ArrestsColumns.DEPARTED_DATE.column_name] <= pd.to_datetime('2025-7-31')))].copy()

d[ArrestsColumns.DEPARTED_DATE.column_name].value_counts()

Departed Date
2025-06-26    1078
2025-07-11    1031
2025-07-18     951
2025-07-04     912
2025-06-05     907
              ... 
2023-11-12       1
2024-05-05       1
2023-11-05       1
2024-09-15       1
2024-04-14       1
Name: count, Length: 692, dtype: int64

In [18]:
# departure country/date == null means they haven't been deported yet
d[ArrestsColumns.DEPARTED_DATE.column_name].isna().value_counts()

Departed Date
False    166439
True     122730
Name: count, dtype: int64

In [19]:
missing_percent_print(d, ArrestsColumns.DEPARTURE_COUNTRY.column_name)
update_country_strings(d, ArrestsColumns.DEPARTURE_COUNTRY.column_name)

d[ArrestsColumns.DEPARTURE_COUNTRY.column_name].value_counts()

Departure Country, 42.46% missing values


Departure Country
MEXICO                   81207
GUATEMALA                23868
HONDURAS                 22287
EL SALVADOR               8131
VENEZUELA                 4730
                         ...  
SAO TOME AND PRINCIPE        1
BAHRAIN                      1
BOTSWANA                     1
AUSTRIA                      1
TURKMENISTAN                 1
Name: count, Length: 176, dtype: int64

In [20]:
# sorted(list(d[~d[ArrestsColumns.DEPARTURE_COUNTRY.column_name].isna()][ArrestsColumns.DEPARTURE_COUNTRY.column_name].unique()))

In [21]:
missing_percent_print(d, ArrestsColumns.GENDER.column_name)

Gender, 0.0% missing values


In [22]:
missing_percent_print(d, ArrestsColumns.APPREHENSION_SITE_LANDMARK.column_name)

Apprehension Site Landmark, 2.162% missing values


In [23]:
# download states table if needed
if not pathlib.Path('states.csv').exists():
    import requests
    with requests.get('https://raw.githubusercontent.com/jasonong/List-of-US-States/refs/heads/master/states.csv') as r:
        with open('states.csv', 'wb') as f:
            f.write(r.content)
states_d = pd.read_csv('states.csv')
states_d['State_Upper'] = states_d['State'].apply(lambda x: x.upper())

In [24]:
apprsl_d = d[~d[ArrestsColumns.APPREHENSION_SITE_LANDMARK.column_name].isna()][[ArrestsColumns.APPREHENSION_SITE_LANDMARK.column_name]].drop_duplicates().copy()
c = ArrestsColumns.APPREHENSION_SITE_LANDMARK.column_name
apprsl_d['repl_val'] = apprsl_d[c].apply(lambda x: x.strip())
# some strings end with full state name instead of abbreviation.
# https://github.com/jasonong/List-of-US-States/blob/master/states.csv

# if the string ends with a full state name, then repale it with the abbreviation
def str_end_state_to_abbr(x):
    for i, r in states_d.iterrows():
        if x.endswith(f" {r['State_Upper']}"):
            # print(x)
            return x.replace(r['State_Upper'], r['Abbreviation'])
    return x

apprsl_d['repl_val'] = apprsl_d['repl_val'].apply(str_end_state_to_abbr)

apprsl_d['repl_val'] = apprsl_d['repl_val'].apply(lambda x: x.upper())
apprsl_d['repl_val'] = apprsl_d['repl_val'].apply(lambda x: x[:-len(' STATE')] if x.endswith(' STATE') else x)
# sometimes, the states at end of strings have a comma and sometimes they don't
# assume if the string ends with an  upper case 2 letter string, then it's a state 
# and we can add a comma to try normalizing it
def add_comma_before_state_abbrev(x):
    m = re.search('[A-Z] ([A-Z][A-Z])$', x)
    if m:
        return f'{x[:-3]}, {m.groups(0)[0]}'
    return x

# 'CAP-String', 'CAP String' => 'CAP - String'
apprsl_d['repl_val'].apply(lambda x: x.replace('CAP-', 'CAP - ', count=1) if x.startswith('CAP-') else x)
apprsl_d['repl_val'].apply(lambda x: x.replace('CAP ', 'CAP - ', count=1) if x.startswith('CAP ') else x)

apprsl_d['repl_val'] = apprsl_d['repl_val'].apply(add_comma_before_state_abbrev)

In [25]:
d = d.merge(apprsl_d, on = ArrestsColumns.APPREHENSION_SITE_LANDMARK.column_name, how = 'left')
d[ArrestsColumns.APPREHENSION_SITE_LANDMARK.column_name] = d['repl_val']
d = d.drop('repl_val', axis = 1).copy()

In [26]:
# d.to_parquet(proc_data / 'arrests_cleaned.parquet')